In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, util, color
import os

In [2]:
def load_ecg_image(image_path): # read image
    image = io.imread(image_path) 
    
    # If the image has an alpha channel (RGBA), remove it
    if image.shape[-1] == 4:
        image = image[:, :, :3]  # Keep only RGB channels
    
    # Convert RGB to grayscale if necessary
    if len(image.shape) == 3:
        image = color.rgb2gray(image)
    
    return (image * 255).astype(np.uint8)  # Normalize to 8-bit

def get(im, i, j): # Return pixel value at (i, j) if within bounds, else return 0 
    if 0 <= i < im.shape[0] and 0 <= j < im.shape[1]:
        return int(im[i, j])
    return 0

def GAP(im, i, j, out, context, N, S, err_for_del, thre):
    """Gradient Adjusted Prediction for ECG Image."""
    # Collect neighboring pixel values for prediction
    In = get(im, i, j-1)
    Iw = get(im, i-1, j)
    Ine = get(im, i+1, j-1)
    Inw = get(im, i-1, j-1)
    Inn = get(im, i, j-2)
    Iww = get(im, i-2, j)
    Inne = get(im, i+1, j-2)

    # Calculate horizontal and vertical gradient magnitudes
    dh = abs(Iw - Iww) + abs(In - Inw) + abs(In - Ine)
    dv = abs(Iw - Inw) + abs(In - Inn) + abs(Ine - Inne)

    # Choose a base predictor based on gradient comparison
    if dv - dh > 80:
        ic = Iw
    elif dv - dh < -80:
        ic = In
    else:
        ic = (Iw + In) / 2 + (Ine - Inw) / 4
        if dv - dh > 32:
            ic = (ic + Iw) / 2
        elif dv - dh > 8:
            ic = (3*ic + Iw) / 4
        elif dv - dh < -32:
            ic = (ic + In) / 2
        elif dv - dh < -8:
            ic = (3*ic + In) / 4

      # Create a binary pattern based on neighbors compared to the predictor
    temp = list(map(lambda x: int(x < ic),[(2*Iw)-Iww,(2*In)-Inn,Iww,Inn,Ine,Inw,Iw,In]))
    B = temp[0] << 7 | temp[1] << 6 | temp[2] << 5 | temp[3] << 4 | temp[4] << 3 | temp[5] << 2 | temp[6] << 1 | temp[7]
    
    # Convert binary pattern to integer (bitmasking to form a context number)
    delt = dh + dv + 2 * abs(err_for_del)
    Qdel = next((k for k, t in enumerate(thre) if delt <= t), 7)
    
    # Final context index for adaptive prediction 
    C = B * Qdel // 2
    
    # Update frequency and error statistics for the context
    N[C] += 1
    S[C] += err_for_del
    if N[C] == 255:
        # Prevent overflow: normalize counts occasionally
        N[C] //= 2
        S[C] //= 2

    # Estimate the error using average past errors in this context
    ed = S[C] // max(N[C], 1)
    
    # Refine prediction using context-based error adjustment
    Itilde = ic + ed
    
    # Save predicted value to output image
    out[i, j] = Itilde

    # Save context index (clamped) to the context map
    context[i, j] = np.clip(C, 0, 255)  # Clamps values between 0 and 255

    # Update the prediction error for use in future predictions
    err_for_del = get(im, i, j) - Itilde

def compress_ecg_image(image_path):
    """Apply CALIC compression to an ECG image."""
    d = load_ecg_image(image_path)# Load and convert the image to grayscale 8-bit

    # Initialize output image and context map with same size as input
    out = np.zeros_like(d, dtype=np.uint8)
    context = np.zeros_like(d, dtype=np.uint8)

    # Thresholds used to quantize local gradient magnitude
    thre = [5, 15, 25, 42, 60, 85, 140]

    # Arrays for maintaining count (N) and error sum (S) for each context
    N = np.zeros((4 * 256,), dtype=np.int32)
    S = np.zeros((4 * 256,), dtype=np.int32)
    
    err_for_del = 0# Initialize the error used for context adaptation

    # Traverse the entire image pixel-by-pixel
    for i in range(d.shape[0]):
        for j in range(d.shape[1]):
            GAP(d, i, j, out, context, N, S, err_for_del, thre)
    
    return out

def save_compressed_image(compressed_image, output_path):
    """Save the compressed image."""
    io.imsave(output_path, compressed_image)

In [3]:
import time
import csv
import os
input_folder = "org_ecg_10sec_resize"
output_folder = "after compress calic"

# Make sure output folder exists
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):  # Adjust extensions if needed
        image_path = os.path.join(input_folder, filename)
        compressed = compress_ecg_image(image_path)
        
        save_path = os.path.join(output_folder, filename)  # Save with same filename
        save_compressed_image(compressed, save_path)